# Post-hoc REPP Analysis Pipeline

Post-hoc analysis of tapping data from psynet experiments using REPP (Rhythm Extraction and Pulse Prediction) beat detection.

**Workflow**: Load participant data → Process audio (mono conversion, 44.1kHz resampling) → Extract stimulus info from CSV → Run REPP beat detection → Generate plots

**Output**: Converted audio (`.wav`), stimulus info (`.json`), and beat detection plots (`.png`) for each participant recording.


In [ ]:
import os
import json
import pandas as pd
import numpy as np
import soundfile as sf
import librosa

import matplotlib.image as mpimg
from matplotlib import pyplot as plt

from post_repp_pipeline import (
    setup_participant_directories,
    process_participant_audio_files,
    run_repp_analysis_for_participant,
)


# Set Parent directory path for Assets dir

In [ ]:
# configure paths
base_dir = r"D:\pyspace\Djembe\psynet\data_2025\November-2025\mali-group1-nov9"     # Set base directory here
output_dir = r"output"

TapTrialMusic_path = os.path.join(base_dir, "data", "TapTrialMusic.csv")
TapTrialMusic_df = pd.read_csv(TapTrialMusic_path)

print("Sub-directories of assets", os.listdir(os.path.join(base_dir, "assets")))
print("Participant Ids:", TapTrialMusic_df['participant_id'].unique())

## Choose sub dir and participant id

In [ ]:
choose_sub_dir = "Task 2"
choose_participant_id = 13

### convert audio and extract stimulus info per participant

In [ ]:
# Setup participant directories
participant_dir, output_participant_dir, participant_audio_fnames = setup_participant_directories(
    base_dir, choose_sub_dir, choose_participant_id, output_dir
)

# Process all audio files: convert audio and extract stimulus info
audio_stim_pairs = process_participant_audio_files(
    participant_dir, 
    output_participant_dir, 
    TapTrialMusic_df,
    overwrite=False  # Set to True to reprocess existing files
)

### Repp Beat Finding Analysis

In [ ]:
# from custom_config import sms_tapping     # see custom_config.py
from repp.config import sms_tapping
from repp.config import ConfigUpdater

config_params= ConfigUpdater.create_config(
    sms_tapping,    # see custom_config.py
    {
        'EXTRACT_THRESH': [0.19, 0.2],
        'EXTRACT_COMPRESS_FACTOR': 1,
        'EXTRACT_FIRST_WINDOW': [18, 18],
        'EXTRACT_SECOND_WINDOW': [26, 60],
        ## TODO: add a parameter that extend the MARKER ERROR THRESHOLD to 20.

    }
)

# Run REPP analysis for all recordings
# %matplotlib inline
results = run_repp_analysis_for_participant(
    audio_stim_pairs,
    output_participant_dir,
    config_params,
    title_plot='Beat Finding Analysis',
    display_plots=True,
    figsize=(14, 12)
)
